In [10]:
# Import necessary libraries
import pandas as pd
import json
import string 

# Load the CSV dataset
df = pd.read_csv("semi_strut.csv")

# Display the first few rows of the dataset
df.head()

,Document ID,Content
0,1,"{\n ""title"": ""Introduction to Python"",\n ""..."
1,2,"{\n ""title"": ""Data Analysis with Pandas"",\n ..."
2,3,"{\n ""title"": ""Web Development with Flask"",\n..."
3,4,"{\n ""title"": ""Machine Learning with Scikit-L..."
4,5,"{\n ""title"": ""Data Visualization with Matplo..."


In [11]:
# Tokenization function to extract terms from the JSON-like content
# Remember to exact both 
# 1 .Extract terms from various fields (title, author)
def tokenize_content(content):
    content_dict = json.loads(content)
    terms = []
    
    # Extract terms from various fields (title, author)
    terms.extend(content_dict.get("title", "").split())
    terms.extend(content_dict.get("author", "").split())
    # terms.extend(content_dict.get("author", "").split())
    return terms
tokenize_content(df["Content"][0])
# 2. apply to all row in panda df , by create new column "Terms"
df["Terms"] = df["Content"].apply(tokenize_content)
print(df.head())


   Document ID                                            Content  \
0            1  {\n   "title": "Introduction to Python",\n   "...   
1            2  {\n   "title": "Data Analysis with Pandas",\n ...   
2            3  {\n   "title": "Web Development with Flask",\n...   
3            4  {\n   "title": "Machine Learning with Scikit-L...   
4            5  {\n   "title": "Data Visualization with Matplo...   

                                               Terms  
0              [Introduction, to, Python, John, Doe]  
1        [Data, Analysis, with, Pandas, Jane, Smith]  
2     [Web, Development, with, Flask, Mike, Johnson]  
3  [Machine, Learning, with, Scikit-Learn, Emily,...  
4  [Data, Visualization, with, Matplotlib, Robert...  


In [12]:
# 4. Implement a preprocessing function that converts terms to lowercase, removes punctuation, and removes common stop words.
    # Create another new column "Terms_preprocessed"
def preprocess_terms(terms):
    # Define a set of common stop words
    stop_words = set([
        "a", "an", "the", "and", "is", "in", "it", "to", "of", "for", "on", "with", "as"
    ])
    
    # Remove punctuation and convert to lowercase
    terms = [term.lower().strip(string.punctuation) for term in terms]
    
    # Remove stop words
    terms = [term for term in terms if term not in stop_words]
    
    return terms

df["Terms_preprocessed"] = df["Terms"].apply(preprocess_terms)

In [14]:
# Initialize an empty inverted index dictionary
# Build the inverted index
# Initialize an empty inverted index dictionary
inverted_index = {}

# Build the inverted index
for index, row in df.iterrows():
    document_id = row["Document ID"]
    terms = row["Terms_preprocessed"]
    
    # Update the inverted index with terms and document IDs
    for term in terms:
        if term not in inverted_index:
            inverted_index[term] = set()
        inverted_index[term].add(document_id)

# Display the inverted index
print(inverted_index)

{'introduction': {1}, 'python': {1}, 'john': {1}, 'doe': {1}, 'data': {2, 5}, 'analysis': {2}, 'pandas': {2}, 'jane': {2}, 'smith': {2}, 'web': {3}, 'development': {3}, 'flask': {3}, 'mike': {3}, 'johnson': {3}, 'machine': {4}, 'learning': {4}, 'scikit-learn': {4}, 'emily': {4}, 'davis': {4}, 'visualization': {5}, 'matplotlib': {5}, 'robert': {5}, 'clark': {5}}


In [21]:
# perform boolean operations on postings lists for Boolean search operations
# 1. "Python" OR "Pandas"
def boolean_or(postings_list_1, postings_list_2):
    merged_postings_list = set()
    for document_id in postings_list_1:
        merged_postings_list.add(document_id)
    for document_id in postings_list_2:
        if document_id not in merged_postings_list:
            merged_postings_list.add(document_id)
    return merged_postings_list

# Get the postings lists for "python" and "pandas"
python_postings_list = inverted_index["python"]
pandas_postings_list = inverted_index["pandas"]

# Perform a Boolean OR operation on the postings lists
merged_postings_list = boolean_or(python_postings_list, pandas_postings_list)

# Print the merged postings list
print(merged_postings_list)
# 2. "Python" AND "data"
def boolean_and(postings_list_1, postings_list_2):
    merged_postings_list = set()
    for document_id in postings_list_1:
        if document_id in postings_list_2:
            merged_postings_list.add(document_id)
    return merged_postings_list

# Get the postings lists for "python" and "data"
python_postings_list = inverted_index["python"]
data_postings_list = inverted_index["data"]

# Perform a Boolean AND operation on the postings lists
merged_postings_list = boolean_and(python_postings_list, data_postings_list)

# Print the merged postings list
print(merged_postings_list)

{1, 2}
set()
